# Notebook used to help create:

- importing data src
- precprocessing data src

In [1]:
%pwd

'C:\\Users\\bjk1y13\\OneDrive - University of Southampton\\MH028_UHS_Weather\\4_Analysis\\EDclassification\\notebooks\\data_cleaning'

In [2]:
import pandas as pd
import numpy as np
from src.data.io import import_pickled_feature_dfs, import_merge_prevday_target_column, import_merge_pickled_target_class

# Import data

In [15]:
features = import_pickled_feature_dfs('../../data/interim/D2_timeseries/', ['EDdaily.pkl', 'IPdaily.pkl', 'callender.pkl'])

In [16]:
features.shape

(2465, 13)

In [17]:
features.tail()

,EDDAILYattendances,EDDAILYadmissions,EDDAILYage_mean,EDDAILYage_75plus,IPDAILYpatients_total,IPDAILYpatients_nonelec,IPDAILYpatients_elec,IPDAILYpatients_daycase,IPDAILYage_mean_elecnonelec,IPDAILYage_75plus_elecnonelec,year,month,dayofweek
,,,,,,,,,,,,,
2018-09-26,446,182,42.9126,62,2,1,1,0,35.5,0,2018,9,Wednesday
2018-09-27,487,188,44.7064,82,2,1,1,0,35.5,0,2018,9,Thursday
2018-09-28,482,168,44.1992,74,2,1,1,0,35.5,0,2018,9,Friday
2018-09-29,463,191,46.4989,84,1,0,1,0,0,0,2018,9,Saturday
2018-09-30,367,157,40.3597,47,1,0,1,0,0,0,2018,9,Sunday


In [18]:
target = pd.read_pickle('../../data/interim/EDooc_class.pkl')

In [19]:
features = target.merge(features, right_index=True, left_index=True)

Remember target does not need to be offset because features were calculated for the previous days records

In [20]:
features_final = features.copy()
# features_final['flag_target'] = features_final['flag_target'].shift(-1)
##### we do shift the EDocc col as this is for the current date in index.
features_final['EDocc'] = features_final['EDocc'].shift(1)

In [24]:
features_final.rename(columns={'EDocc':'TARGET_prevday'},inplace=True)

In [25]:
features_final.flag_target.value_counts()

0    2094
1     371
Name: flag_target, dtype: int64

In [26]:
features_final.head()

,TARGET_prevday,flag_target,EDDAILYattendances,EDDAILYadmissions,EDDAILYage_mean,EDDAILYage_75plus,IPDAILYpatients_total,IPDAILYpatients_nonelec,IPDAILYpatients_elec,IPDAILYpatients_daycase,IPDAILYage_mean_elecnonelec,IPDAILYage_75plus_elecnonelec,year,month,dayofweek
dt_date,,,,,,,,,,,,,,,
2012-01-01,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,1,Sunday
2012-01-02,55.750000,0,377,151,40.0186,59,1185,1001,141,43,57.3814,441,2012,1,Monday
2012-01-03,38.541667,0,350,139,42.2114,44,1228,1037,141,50,57.4292,453,2012,1,Tuesday
2012-01-04,40.208333,0,356,125,45.1011,58,1564,1074,208,282,55.5352,475,2012,1,Wednesday
2012-01-05,32.375000,0,284,92,42.5035,49,1535,1066,241,228,55.4072,482,2012,1,Thursday


IP occ data after 2018-5-31 is not valid. NOTE: must check if that is same in RAW.

## check and remove nans

In [29]:
features_final.isnull().sum()

TARGET_prevday                   1
flag_target                      0
EDDAILYattendances               1
EDDAILYadmissions                1
EDDAILYage_mean                  1
EDDAILYage_75plus                1
IPDAILYpatients_total            1
IPDAILYpatients_nonelec          1
IPDAILYpatients_elec             1
IPDAILYpatients_daycase          1
IPDAILYage_mean_elecnonelec      1
IPDAILYage_75plus_elecnonelec    1
year                             0
month                            0
dayofweek                        0
dtype: int64

In [33]:
features_final.dropna(inplace=True)

# Processing

#### split test and train sets

In [34]:
from src.data.preprocess import make_timeseries_test_train_splits

In [35]:
X_train, X_test, y_train, y_test = make_timeseries_test_train_splits(features_final, 'flag_target', 365)

DATA POINTS:
orig size 2464
training:  2099
testing:  365


In [36]:
from src.data.preprocess import get_variable_types_lists, check_for_catagorical_type_difference_between_train_test, change_feature_types_to_numeric
    

# num_features, cat_features, bin_features = get_variable_types_lists(X_train)

In [37]:
X_train.columns

Index(['TARGET_prevday', 'EDDAILYattendances', 'EDDAILYadmissions',
       'EDDAILYage_mean', 'EDDAILYage_75plus', 'IPDAILYpatients_total',
       'IPDAILYpatients_nonelec', 'IPDAILYpatients_elec',
       'IPDAILYpatients_daycase', 'IPDAILYage_mean_elecnonelec',
       'IPDAILYage_75plus_elecnonelec', 'year', 'month', 'dayofweek'],
      dtype='object')

In [38]:
cat_features = ['month','dayofweek']
num_features = list(set(X_train.columns) - set(cat_features))
bin_features = []

#### find problem columns that will crash preprocessing pipleine when converting test data

In [39]:
problem_col_list = check_for_catagorical_type_difference_between_train_test(X_train, X_test, cat_features)

In [40]:
problem_col_list

Treat problem features as numerical so that process can continue.


- use sklearn pipelines to clean


In [41]:
# version 19.2
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder 

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from src.data.preprocess import DataFrameSelector, MakeBooleanAnInteger

for col_name in cat_features:
    values_list = X_train[col_name].unique()
    for value in values_list:
        print(col_name + '_' + str(value))

In [42]:
#### failed to get this to work as kept doubling size of output array. Didnt matter if get_col_list_for_after_pipeline fundtion was inside or outside of DataFrameReform class.


from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameReform(BaseEstimator,TransformerMixin):
    " Takes numpy array and forms into dataframe with column names."
    def __init__(self, new_features_list):
        self.new_features_list = new_features_list
    def fit(self, X, y=None):
        return self
    def transform(self,X):        
        return(pd.DataFrame(X, columns = self.new_features_list))

In [43]:
num_pipeline = Pipeline([
    ('selector',DataFrameSelector(num_features)),
    ('feature_filter',SelectKBest(f_classif,k='all')),
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler',StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_features)),
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('OneHot_encoder',OneHotEncoder(sparse=False))
])

bin_pipeline = Pipeline([
    ('selector',DataFrameSelector(bin_features)),
    ('boolean_conversion',MakeBooleanAnInteger()),
    ('imputer',SimpleImputer(strategy='most_frequent')),
    #('encoding',LabelEncoder()),
])

# df_reform_pipeline = Pipeline([
#     ('reform_df', DataFrameReform(new_features_list))
# ])

#### create list of pipelines to include

pipes_list = [
    ('num_pipeline',num_pipeline),
  ('cat_pipeline',cat_pipeline),
#   ('bin_pipeline',bin_pipeline),
#     ('df_reform_pipeline', df_reform_pipeline)
]

full_pipeline = FeatureUnion(transformer_list=pipes_list
)


X_trainT = full_pipeline.fit_transform(X_train,y_train)
X_testT = full_pipeline.transform(X_test)

In [44]:
X_train.shape

(2099, 14)

In [45]:
X_trainT.shape

(2099, 31)

In [46]:
def get_col_list_for_after_pipeline(X, num_features, cat_features, bin_features):
    cat_features_new = []
    #### get new names for cat_features - must do here, otherwise when call fit transform the cat_features_new is appedned too twice.
#     print('TRANSFORM METHOD')
    for col_name in cat_features:
#         print('CAT FEATURES LOOP', col_name)
        values_list = X[col_name].unique()
        for value in values_list:
            cat_features_new.append(col_name + '_' + str(value))
#         print(cat_features_new)
    
    columns_list = num_features + cat_features_new + bin_features
#     print(len(columns_list))
    return(columns_list)

new_features_list = get_col_list_for_after_pipeline(X_train, num_features, cat_features, bin_features)

In [47]:
def reform_df(X, new_features_list):
    return(pd.DataFrame(X, columns= new_features_list))

X_trainT = reform_df(X_trainT, new_features_list)
X_testT = reform_df(X_testT, new_features_list)

In [48]:
X_testT.shape

(365, 31)

In [49]:
X_trainT.shape

(2099, 31)

In [50]:
X_testT.shape

(365, 31)

# Save preprocessed out as pkl

In [52]:
def save_pickle_preprocessed_data(path, X_train, X_test, y_train, y_test):
    """
    Take prepared data which has been split into Train/Test and has been scaled/blanks filled/...., and save to pickle files at specified location.
    
    Input
    =====
    path, str, to folder where data should be saved. Must end in /

    X_train/X_test/y_train,/y_test, dataframes, conatining data.
    
    Ouput
    =====
    
    """
    
    #### create folder with versioned name etc.(future dev)
    
    pd.to_pickle(X_train, path + 'X_train.pkl')
    pd.to_pickle(X_test, path + 'X_test.pkl')
    pd.to_pickle(y_train, path + 'y_train.pkl')
    pd.to_pickle(y_test, path + 'y_test.pkl')
    
    return

save_pickle_preprocessed_data('../../data/processed/D2/', X_trainT, X_testT, y_train, y_test)

NOTE: consider creating log for preprocessing information in order for repeatability in future.